Copyright (c) Microsoft Corporation.

Licensed under the MIT License.

In [ ]:
import ast

# convert string parameter to list of strings
delta_load_user_group= ast.literal_eval(delta_load_user_group)
initial_load_user_group = ast.literal_eval(initial_load_user_group)

In [ ]:
base_path = f"abfss://{file_system_name}@{data_lake_account_name}.dfs.core.windows.net/"

if initialLoad == 'true':
    user_group = initial_load_user_group
    user_group_name = initial_load_user_group_name
else: 
    user_group = delta_load_user_group
    user_group_name = delta_load_user_group_name
    

messagesPath = base_path +"o365data/" + folder_name + "/" + user_group_name + "/"

In [ ]:
import requests
import msal
import logging


linked_service = ""
keyvault_name = ""

authority = mssparkutils.credentials.getSecret(linkedService=linked_service,akvName=keyvault_name,secret='')
client_id = mssparkutils.credentials.getSecret(linkedService=linked_service,akvName=keyvault_name,secret='')
secret = mssparkutils.credentials.getSecret(linkedService=linked_service,akvName=keyvault_name,secret='')


scope = ["https://graph.microsoft.com/.default"]
base_endpoint = "https://graph.microsoft.com/v1.0"


app = msal.ConfidentialClientApplication(client_id, authority=authority,client_credential=secret)

result = None
result = app.acquire_token_silent(scope, account=None)

if not result:
    logging.info("No suitable token exists in cache. Let's get a new one from AAD.")
    result = app.acquire_token_for_client(scope)

if "access_token" not in result:
    print(result.get("error"))
    print(result.get("error_description"))
    print(result.get("correlation_id"))  # You may need this when reporting a bug

headers = {'Authorization': 'Bearer ' + result['access_token']}


def get_group_members(base_endpoint, headers, group_id):
    endpoint = base_endpoint + '/groups/' + group_id + '/members' 
    graph_data = requests.get(endpoint,headers=headers,).json()
    return(graph_data)

def delete_group_members(base_endpoint, headers, group_id,member_ids):
    for member_id in member_ids:
        endpoint = base_endpoint + '/groups/' + group_id + '/members/' + member_id + '/$ref'
        response = requests.delete(endpoint, headers=headers)
    return(response)

def add_group_members(base_endpoint, headers, group_id,members_data):
    endpoint = base_endpoint + '/groups/' + group_id
    response = requests.patch(endpoint,json=members_data, headers=headers)
    return(response)

In [ ]:
from datetime import datetime,timezone
from pyspark.sql.functions import col

def get_date_range(initialLoad,base_endpoint,headers,user_group,messagesPath, initial_load_user_group_name): 
    from_date = '2020-01-01 00:00:01'

    date_range = ''

    if initialLoad == 'true':
        #add members from config file to AD group
        

        df = spark.read.format("cosmos.oltp")\
            .option("spark.synapse.linkedService", "CosmosDB")\
            .option("spark.cosmos.container", "users")\
            .load()
        
         # Uncomment this if your Linked Service is enabled with a private endpoint 
        #df = spark.read.format("cosmos.oltp")\
        #    .option("spark.cosmos.useGatewayMode", True)\
        #    .option("spark.synapse.linkedService", "CosmosDB")\
        #    .option("spark.cosmos.container", "users")\
        #    .load()

        df = df.filter(col('initialload_completion_flag') == 0)
        member_ids = df.select("user_id").rdd.flatMap(lambda x: x).collect()

        if len(member_ids) == 0:
            return('Success')
        
        # get start and end dates
        from_date = str(datetime.strptime(from_date, "%Y-%m-%d %H:%M:%S"))

    
    else:
        #check if there are any members in delta group
        df_members = get_group_members(base_endpoint,headers,delta_load_user_group[0])
        mcount = len(df_members['value'])
        
        # what to do if memer count is zero?
        if mcount == 0:
            return('Success')

         # get start and end dates
        try: 
            folders = mssparkutils.fs.ls(messagesPath)
            if len(folders) == 0: 
            # delta load folder is empty only the first time delta load is run for each user group 
                user_group_name = initial_load_user_group_name
                messagesPath = base_path +"o365data/" + folder_name + "/" + user_group_name + "/"
        except: 
            user_group_name = initial_load_user_group_name
            messagesPath = base_path +"o365data/" + folder_name + "/" + user_group_name + "/"
        
        folders = mssparkutils.fs.ls(messagesPath)
        foldernames = [f.name for f in folders]
        foldernames.sort(reverse=True)
        from_date = foldernames[0].split('_')[1]
        from_date = str(datetime.strptime(from_date, "%Y-%m-%d-%H-%M-%S"))
        if from_date == None:
            from_date = '2020-01-01 00:00:00'

    to_date = datetime.strptime(str(datetime.now()), "%Y-%m-%d %H:%M:%S.%f")
    to_date = to_date.strftime("%Y-%m-%d %H:%M:%S")

    date_range = from_date + '_' + to_date
    return(date_range)


In [ ]:
date_range = get_date_range(initialLoad,base_endpoint,headers,user_group,messagesPath, initial_load_user_group_name)

In [ ]:
mssparkutils.notebook.exit(date_range)